In [1]:
import import_ipynb
import copy
from builtins import range
import cubiecube as QB
import corner as Co
import edge as Ed
import facelet as Fc
import facecube as FC
import numpy as np

importing Jupyter notebook from cubiecube.ipynb
importing Jupyter notebook from corner.ipynb
importing Jupyter notebook from facelet.ipynb
importing Jupyter notebook from edge.ipynb
importing Jupyter notebook from facecube.ipynb
importing Jupyter notebook from color.ipynb


In [64]:
def count_RB(not_picked, candi):
    count = 0
    for i in range(len(not_picked)):
        if not not_picked[i] and candi < i:
            count += 1
    return count 
    
    
def Corner_coordinateToPermutation(a = [0,0,0,0,0,0,0]):
    b = a[::-1]
    ans = [0 for i in range(8)]
    ans[0] = 7 - b[0]
    not_picked = [True  for i in range(8)]
    not_picked[ans[0]] = False
    for i in range(1, len(ans)-1):
        for candidate in range(7-b[i]+1):
            RB = count_RB(not_picked, candidate)
            LB = b[i]
            #print("RB=", RB, " LB=", LB)
            if not_picked[candidate] and LB + RB + candidate == 7:
                ans[i] = candidate
                not_picked[ans[i]] = False
    for i in range(len(not_picked)):
        if not_picked[i]:
            ans[-1] = i

    return ans[::-1]

def Edge_coordinateToPermutation(a = [0,0,0,0,0,0,0,0,0,0,0]):
    a = list(a)
    b = a[::-1]
    ans = [0 for i in range(12)]
    ans[0] = 11 - b[0]
    not_picked = [True  for i in range(12)]
    not_picked[ans[0]] = False
    for i in range(1, len(ans)-1):
        for candidate in range(11-b[i]+1):
            RB = count_RB(not_picked, candidate)
            LB = b[i]
            # RB : 右邊比他大的數量  LB : 左邊比他大的數量
            if not_picked[candidate] and LB + RB + candidate == 11:
                ans[i] = candidate
                not_picked[ans[i]] = False
    for i in range(len(not_picked)):
        if not_picked[i]:
            ans[-1] = i
    return ans[::-1]

class CoordinateCube():
    def __init__(self, cp = 0, co =0, ep = 0, eo = 0):
        # we ignore the DRB entry since it can be determined by other terms.
        """
        We use numbers to record the orientation and permutation.
        
        By expressing co and eo in trinary and binary base respectively, we'll get the first 7 and 11 orientations.
        The other orientation can be determined by whether it's divided by 3 or 2
        
        By expressing cp and ep in variable base, we'll get the permutation except the first one
        The first can be easily determined by checking the other element.
        
        0 <= co < 3^7
        0 <= eo < 2^11
        0 <= cp < 8!
        0 <= ep < 12!
        
        """
        self.cp = cp
        self.co = co
        self.ep = ep
        self.eo = eo
    
    def __str__(self):
        return str(self.to_cubiecube())
    
    def to_2dstring(self):
        return self.to_cubiecube().to_2dstring()
    
     
    def to_cubiecube(self):
        
        cp = []
        pre_cp = []               # we need to interpret the digits in variable base into permutation
        n_cp = self.cp
        for i in range(1,8):        # assign the permutations except URF
            pre_cp.append(n_cp%np.math.factorial(i+1))
        pre_cp = np.array(pre_cp)
        pre_cp[1:] =  pre_cp[1:8] - pre_cp[:-1]
        pre_cp = pre_cp / np.array([ np.math.factorial(i) for i in range(1, 8) ])
        cp = Corner_coordinateToPermutation(pre_cp.astype(int))
        
        
        ep = []
        pre_ep = []               # we need to interpret the digits in variable base into permutation
        n_ep = self.ep
        for i in range(1,12):        # assign the permutations except URF
            pre_ep.append(n_ep%np.math.factorial(i+1))
        pre_ep = np.array(pre_ep)
        pre_ep[1:] =  pre_ep[1:12] - pre_ep[:-1]
        pre_ep = pre_ep / np.array([ np.math.factorial(i) for i in range(1, 12) ])
        ep = Edge_coordinateToPermutation(pre_ep.astype(int))

        co = [0 for i in range(8)]
        n_co = self.co
        count_co = 0
        for i in range(7):        # assign first 7 orientations
            count_co += n_co%3
            co[6 - i] = n_co%3
            n_co //= 3
        co[7] = (-count_co)%3 # sum of all entries must be divided by 3
        co = co[::-1]
        print(co)
        
        eo = [0 for i in range(12)]
        n_eo = self.eo
        count_eo = 0
        for i in range(11):       # assign first 11 orientations
            count_eo += n_eo%2
            eo[11 - i] = n_eo%2
            n_eo //= 2
        eo[11] = count_eo%2 # sum of all entries must be divided by 2
        eo = eo[::-1]
        
        Q = QB.CubieCube(cp, np.array(co), ep, np.array(eo))
        return Q